In [115]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/Tritangent_plane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [83]:
var('nv')
K1.<gamma> = NumberField(nv^4-4*nv^2+16)

In [84]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(K1)

In [85]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [86]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [87]:
line = Line([y, z])

In [88]:
eqn_SE1 = P(cubic_new.subs({b:-(c*c+e*f)/c }) )
sing_cubics_SE1 = P(sing_cubics.subs({b:-(c*c+e*f)/c }))

In [89]:
eqn_SE3 = P(2*eqn_SE1.subs({d:(3*e*f - c^2 + c*f + c*e)/(2*c)}) )
sing_cubics_SE3 = P(sing_cubics_SE1.subs({d:(3*e*f - c^2 + c*f + c*e)/(2*c)}))

In [90]:
eqn_SE9 = P(eqn_SE3.subs({e : (-1/2*gamma^2 + 1)*c}) /c)
sing_cubics_SE9 = P(sing_cubics_SE3.subs({e : (-1/2*gamma^2 + 1)*c}))

In [91]:
SE9 = Cubic(eqn_SE9, line, sing_cubics_SE9.factor())

In [92]:
SE9.eckardt_points

[(0, 0, 0, 1),
 (0, 1, 1, 0),
 ((1/6*gamma^2 - 1/3), (1/6*gamma^2 - 1/3), 1, -1),
 ((-1/8*gamma^2 + 1/2)*c^2 + (1/8*gamma^2 - 1/2)*c*f, c^2, c*f, (1/8*gamma^2 - 1/2)*c*f + (-1/8*gamma^2 + 1/2)*f^2),
 ((-1/12*gamma^2 + 1/6)*c^2 + (1/12*gamma^2 - 1/6)*c*f, (-1/12*gamma^2 + 2/3)*c^2, (-1/12*gamma^2 + 2/3)*c*f, (-1/8*gamma^2 + 1/2)*c^2 + (1/6*gamma^2 - 5/6)*c*f + (-1/24*gamma^2 + 1/3)*f^2),
 ((5/84*gamma^2 - 4/21)*c + (-5/84*gamma^2 + 4/21)*f, (-1/21*gamma^2 - 1/21)*c + (-1/21*gamma^2 - 1/21)*f, (-1/14*gamma^2 + 3/7)*c + (-1/42*gamma^2 - 11/21)*f, (-1/28*gamma^2 - 2/7)*c + (1/28*gamma^2 + 2/7)*f),
 ((1/8*gamma^2)*c^2 + (-1/8*gamma^2)*c*f, -1/2*c^2 - 1/2*c*f, (-1/4*gamma^2 + 1/2)*c^2 + (1/4*gamma^2 - 3/2)*c*f, (1/4*gamma^2 - 1/2)*c^2 + (-3/8*gamma^2 + 1)*c*f + (1/8*gamma^2 - 1/2)*f^2),
 (0, (-1/4*gamma^2)*c, (-1/4*gamma^2)*c, (1/4*gamma^2 - 1)*c + (-1/4*gamma^2 + 1)*f),
 (1/2*c, 1/2*c, (-1/4*gamma^2 + 1/2)*c, -1/2*f)]

In [14]:
%time adm_SE9 = SE9.find_admissible_projectivities()

CPU times: user 7.71 s, sys: 302 ms, total: 8.01 s
Wall time: 1min


In [15]:
len(adm_SE9)

1296

In [16]:
%time simm_SE9 = SE9.find_simmetries(adm_SE9)

CPU times: user 4.72 s, sys: 166 ms, total: 4.89 s
Wall time: 5.47 s


In [17]:
len(simm_SE9)

54

In [116]:
%time conds = find_conditions_for_subfamilies(SE9, sing_cubics_SE9, adm_SE9, simm_SE9)

CPU times: user 3min 54s, sys: 3.02 s, total: 3min 57s
Wall time: 4min 3s


In [117]:
conds

[[13*c + (-gamma^3 - gamma^2 + 3*gamma + 3)*f],
 [4*c + (-gamma^3 + 2*gamma^2 - 4)*f],
 [13*c + (gamma^3 - gamma^2 - 3*gamma + 3)*f],
 [52*c + (gamma^3 - 4*gamma^2 + 12*gamma + 4)*f],
 [52*c + (-gamma^3 - 4*gamma^2 - 12*gamma + 4)*f],
 [4*c + (gamma^3 + 2*gamma^2 - 4)*f]]

In [101]:
sol = solve_linear_system(conds[1], [c], [f])

In [113]:
sing_cubics_SE9.subs({c:sol[0], f:sol[1]}).factor()

((224375739696661926087313346737718951936*gamma^3 - 518173574853956650423415033802562469888*gamma^2 + 1036347149707913300846830067605124939776)) * f^31

In [102]:
sing_cubics_SE9_1 = f
SE9_1 = Cubic(remove_sing_factors(SE9.eqn.subs({c:sol[0], f:sol[1]}), sing_cubics_SE9_1.factor()), line, sing_cubics_SE9_1.factor())

In [114]:
SE9_1

(-8*gamma^2 + 16)*x^2*y + (8*gamma^2 - 16)*x*y^2 + (8*gamma^2 - 16*gamma)*x^2*z + (-4*gamma^3 - 4*gamma^2 + 24*gamma - 16)*x*y*z + (4*gamma^2 - 8*gamma)*x*z^2 + (-4*gamma^3 + 8*gamma^2 + 16*gamma - 32)*x*y*t + (-4*gamma^2 - 8*gamma + 16)*y^2*t + (4*gamma^3 - 4*gamma^2 - 8*gamma + 16)*y*z*t + (4*gamma^3 - 8*gamma^2 + 16)*y*t^2

In [111]:
len(SE9_1.eckardt_points)

9

In [103]:
%time adm_SE9_1 = SE9_1.find_admissible_projectivities()

CPU times: user 4.3 s, sys: 393 ms, total: 4.69 s
Wall time: 36.2 s


In [104]:
len(adm_SE9_1)

1296

In [105]:
simm_SE9_1 = SE9_1.find_simmetries(adm_SE9_1)

In [106]:
len(simm_SE9_1)

108

In [107]:
%time conds_1 = find_conditions_for_subfamilies(SE9_1, sing_cubics_SE9_1, adm_SE9_1, simm_SE9_1)

CPU times: user 4min 38s, sys: 1.78 s, total: 4min 40s
Wall time: 4min 48s


In [108]:
conds_1

[]